In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
import sys
os.getcwd()
sys.path.append("C:/SQL_PW/")
import config

### Store CSV into DataFrame

In [2]:
csv_file = "Resources/snap_national_data.csv"
snap_df = pd.read_csv(csv_file)
snap_df.head()

,CODEF,COFIPS,CONAME,FIPS,FIPSTXT,METRO,NUMPOV00,NUMPOV06,NUMPOV07,NUMPOV10,...,PRGNUM11,PRGNUM89,PRGNUM93,PRGNUM95,PRGNUM97,PRGNUM98,PRGNUM99,STATE_CD,STFIPS,URBCODE
0,CEN_BEA,25,"Cass County, NE",31025,31025,1,1574,1544,1604,1.787,...,-9991.0,1016.0,1441.0,1362.0,1374.0,1328.0,992.0,NE,31,2
1,CEN_BEA,69,"Garden County, NE",31069,31069,0,325,307,277,321.000,...,-9991.0,75.0,130.0,9.0,22.0,125.0,95.0,NE,31,9
2,CEN_BEA,73,"Gosper County, NE",31073,31073,0,163,184,169,191.000,...,-9991.0,30.0,37.0,9.0,13.0,42.0,32.0,NE,31,9
3,CEN_BEA,113,"Logan County, NE",31113,31113,0,89,83,79,96.000,...,-9991.0,32.0,9.0,3.0,0.0,32.0,29.0,NE,31,9
4,CEN_BEA,137,"Phelps County, NE",31137,31137,0,833,898,897,894.000,...,-9991.0,416.0,458.0,436.0,421.0,431.0,394.0,NE,31,7


### Clean DataFrame

In [3]:
# Keep only columns meaningful to this analysis
new_snap_df = snap_df[['CODEF', 'STATE_CD', 'POP07', 'POP10',\
                       'NUMPOV07', 'NUMPOV10', \
                       'PRGNUM07', 'PRGNUM10', \
                       'PRGBEN07', 'PRGBEN10']].copy()


In [4]:
# Convert columns to 'object' type to prevent pandas from forcing datatype to float
new_snap_df['POP07']= new_snap_df['POP07'].astype(object)
new_snap_df['POP10']= new_snap_df['POP10'].astype(object)

new_snap_df['NUMPOV07']= new_snap_df['NUMPOV07'].astype(object)
new_snap_df['NUMPOV10']= new_snap_df['NUMPOV10'].astype(object)

new_snap_df['PRGNUM07']= new_snap_df['PRGNUM07'].astype(object)
new_snap_df['PRGNUM10']= new_snap_df['PRGNUM10'].astype(object)

new_snap_df.head(10)

,CODEF,STATE_CD,POP07,POP10,NUMPOV07,NUMPOV10,PRGNUM07,PRGNUM10,PRGBEN07,PRGBEN10
0,CEN_BEA,NE,25520,25291,1604,1.787,1034,1727,1083.9,2520.0
1,CEN_BEA,NE,1843,2065,277,321,114,187,119.5,272.9
2,CEN_BEA,NE,1960,2047,169,191,61,95,63.9,138.6
3,CEN_BEA,NE,741,770,79,96,19,68,19.9,99.2
4,CEN_BEA,NE,9134,9199,897,894,532,700,557.7,1021.4
5,CEN_BEA,NE,3574,3809,473,441,185,342,193.9,499.0
6,CEN_BEA,NJ,446314,449149,21750,25.008,12073,22811,14080.5,37783.2
7,CEN_BEA,NJ,589377,635294,81698,103.576,57562,91947,67133.2,152297.4
8,CEN_BEA,NJ,487067,50186,64765,76.91,52689,82573,61450.0,136770.7
9,CEN_BEA,NJ,519738,537661,41463,58.007,22610,38268,26369.5,63385.6


In [5]:
# Drop all NaN cells. If time permitted for this project, it'd make more sense to use median 
# to replace the NaN value 
new_snap_df = new_snap_df.dropna(how='any')
new_snap_df.head()

,CODEF,STATE_CD,POP07,POP10,NUMPOV07,NUMPOV10,PRGNUM07,PRGNUM10,PRGBEN07,PRGBEN10
0,CEN_BEA,NE,25520,25291,1604,1.787,1034,1727,1083.9,2520.0
1,CEN_BEA,NE,1843,2065,277,321,114,187,119.5,272.9
2,CEN_BEA,NE,1960,2047,169,191,61,95,63.9,138.6
3,CEN_BEA,NE,741,770,79,96,19,68,19.9,99.2
4,CEN_BEA,NE,9134,9199,897,894,532,700,557.7,1021.4


In [6]:
# Drop all negative values. If time permitted for this project, it'd make more sense to use median 
# to replace the NaN value
new_snap_df = new_snap_df.drop(new_snap_df[new_snap_df.POP07 <0].index)
new_snap_df = new_snap_df.drop(new_snap_df[new_snap_df.POP10 <0].index)

new_snap_df = new_snap_df.drop(new_snap_df[new_snap_df.NUMPOV10 <0].index)
new_snap_df = new_snap_df.drop(new_snap_df[new_snap_df.NUMPOV07 <0].index)

new_snap_df = new_snap_df.drop(new_snap_df[new_snap_df.PRGNUM10 <0].index)
new_snap_df = new_snap_df.drop(new_snap_df[new_snap_df.PRGNUM07 <0].index)

new_snap_df = new_snap_df.drop(new_snap_df[new_snap_df.PRGBEN10 <0].index)
new_snap_df = new_snap_df.drop(new_snap_df[new_snap_df.PRGBEN07 <0].index)

new_snap_df.head(10)

,CODEF,STATE_CD,POP07,POP10,NUMPOV07,NUMPOV10,PRGNUM07,PRGNUM10,PRGBEN07,PRGBEN10
0,CEN_BEA,NE,25520,25291,1604,1.787,1034,1727,1083.9,2520.0
1,CEN_BEA,NE,1843,2065,277,321,114,187,119.5,272.9
2,CEN_BEA,NE,1960,2047,169,191,61,95,63.9,138.6
3,CEN_BEA,NE,741,770,79,96,19,68,19.9,99.2
4,CEN_BEA,NE,9134,9199,897,894,532,700,557.7,1021.4
5,CEN_BEA,NE,3574,3809,473,441,185,342,193.9,499.0
6,CEN_BEA,NJ,446314,449149,21750,25.008,12073,22811,14080.5,37783.2
7,CEN_BEA,NJ,589377,635294,81698,103.576,57562,91947,67133.2,152297.4
8,CEN_BEA,NJ,487067,50186,64765,76.91,52689,82573,61450.0,136770.7
9,CEN_BEA,NJ,519738,537661,41463,58.007,22610,38268,26369.5,63385.6


In [46]:
# Test Block
# import numbers
# new_snap_df['POP10'] = new_snap_df['POP10'].apply(lambda x: x if isinstance(x, numbers.Integral) else x*1000)
# new_snap_df.head()

In [48]:
# Test Block
# In population columns, when decimal point presents, multiply the number by 1000
#new_snap_df['POP06'] = new_snap_df['POP06'].apply(lambda x: x * 1000 if '.' in str(x) else x)
#new_snap_df['POP07'] = new_snap_df['POP07'].apply(lambda x: x * 1000 if '.' in str(x) else x)
#new_snap_df['POP10'] = new_snap_df['POP10'].apply(lambda x: x * 1000 if '.' in str(x) else x)

#new_snap_df['NUMPOV06'] = new_snap_df['NUMPOV06'].apply(lambda x: x * 1000 if '.' in str(x) else x)
#new_snap_df['NUMPOV07'] = new_snap_df['NUMPOV07'].apply(lambda x: x * 1000 if '.' in str(x) else x)

#new_snap_df['PRGNUM06'] = new_snap_df['PRGNUM06'].apply(lambda x: x * 1000 if '.' in str(x) else x)
#new_snap_df['PRGNUM07'] = new_snap_df['PRGNUM07'].apply(lambda x: x * 1000 if '.' in str(x) else x)

#new_snap_df.head()

In [7]:
# Convert the population columns to integer
new_snap_df['POP07'] = new_snap_df['POP07'].astype(int)
new_snap_df['POP10'] = new_snap_df['POP10'].astype(int)

new_snap_df['NUMPOV07'] = new_snap_df['NUMPOV07'].astype(int)
new_snap_df['NUMPOV10'] = new_snap_df['NUMPOV10'].astype(int)

new_snap_df['PRGNUM07'] = new_snap_df['PRGNUM07'].astype(int)
new_snap_df['PRGNUM10'] = new_snap_df['PRGNUM10'].astype(int)

new_snap_df.head()

,CODEF,STATE_CD,POP07,POP10,NUMPOV07,NUMPOV10,PRGNUM07,PRGNUM10,PRGBEN07,PRGBEN10
0,CEN_BEA,NE,25520,25291,1604,1,1034,1727,1083.9,2520.0
1,CEN_BEA,NE,1843,2065,277,321,114,187,119.5,272.9
2,CEN_BEA,NE,1960,2047,169,191,61,95,63.9,138.6
3,CEN_BEA,NE,741,770,79,96,19,68,19.9,99.2
4,CEN_BEA,NE,9134,9199,897,894,532,700,557.7,1021.4


In [8]:
snap_by_state_df = new_snap_df.groupby(['STATE_CD']).agg({'POP07':'sum', 'POP10':'sum', \
                                                          'NUMPOV07':'sum', 'NUMPOV10':'sum', \
                                                          'PRGNUM07':'sum', 'PRGNUM10':'sum', \
                                                          'PRGBEN07':'sum', 'PRGBEN10':'sum'})

snap_by_state_df


,POP07,POP10,NUMPOV07,NUMPOV10,PRGNUM07,PRGNUM10,PRGBEN07,PRGBEN10
STATE_CD,,,,,,,,
AK,667605,616400,63264,5539,50158,70668,85135.0,154949.1
AL,4637904,4369309,750198,848,561025,763629,613789.0,1309165.1
AR,2842194,2584673,483512,1297,376869,433327,420933.0,696050.0
AZ,2489279,2552967,380615,474,303030,476818,359810.8,743534.1
CA,10951876,10695211,1407311,2439,780755,1208039,921819.0,2021779.0
CO,4842259,4805513,548521,9067,249457,409183,309039.3,750026.6
CT,3488633,3406217,269045,346,221603,371357,263826.4,629511.9
DC,586409,604453,95441,107,88504,131366,106090.0,202559.0
DE,864896,899769,86654,103,72404,102414,80534.0,197796.2


In [77]:
snap_by_state_df = snap_by_state_df[~snap_by_state_df.index.str.contains('"')]

In [78]:
snap_by_state_df

,POP07,POP10,NUMPOV07,NUMPOV10,PRGNUM07,PRGNUM10,PRGBEN07,PRGBEN10
STATE_CD,,,,,,,,
AK,662498,683946,69231,63264,48620,5309,83255.4,132187.5
AL,4597688,4708708,743558,750198,543923,747,581681.6,1060197.0
AR,2815097,2889450,471166,483512,381607,1111,411510.8,632558.5
AZ,6192100,6595778,860354,876120,537075,978,622849.0,1483094.5
CA,35979208,36961664,4686706,4445390,2032370,3897,2401041.6,4803812.0
CO,4753044,5024748,539330,548521,252441,13574,317681.5,610383.7
CT,3485162,3518288,281078,269045,211299,313,242617.1,513387.8
DC,583978,599657,99671,95441,87027,114,103219.0,171137.0
DE,853022,885122,88749,86654,66247,106,70629.0,141422.0


In [80]:
snap_by_state_df.reset_index('STATE_CD', inplace=True)

### Connect to local database

In [10]:
rds_connection_string = f"root:{config.password}@127.0.0.1:3306/snap_db"
engine = create_engine(f'mysql://{rds_connection_string}')

engine.table_names()

['snap_participants']

### Use pandas to load csv converted DataFrame into database

In [11]:
snap_by_state_df.to_sql(name='snap_participants', con=engine, if_exists='replace', index=False)

pd.read_sql_query('select * from snap_participants', con=engine).head()

,STATE_CD,POP07,POP10,NUMPOV07,NUMPOV10,PRGNUM07,PRGNUM10,PRGBEN07,PRGBEN10
0,AK,662498,683946,69231,63264,48620,5309,83255.4,132187.5
1,AL,4597688,4708708,743558,750198,543923,747,581681.6,1060197.0
2,AR,2815097,2889450,471166,483512,381607,1111,411510.8,632558.5
3,AZ,6192100,6595778,860354,876120,537075,978,622849.0,1483094.5
4,CA,35979208,36961664,4686706,4445390,2032370,3897,2401041.6,4803812.0
